In [153]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

In [154]:
#Abrimos el df registrados con encoding 
df= pd.read_csv("../data/03_registrados_cod.csv", index_col=0)
df.head(2)

,año,sensacion_termica,humedad,viento,registrados,mes_map,estacion_map,dia_semana_map,clima_map,vacaciones_map,cod_SI
0,0,18.18125,80.5833,10.749882,654,0,0,0,1,0,0
1,0,17.68695,69.6087,16.652113,670,0,0,0,1,1,1


In [155]:
# separamos nuestro dataframe en X e y

X = df.drop("registrados", axis = 1)
y = df["registrados"]

In [156]:
# dividimos nuestros datos en train y test para poder evaluar la validez de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [157]:
# creamos el objeto del modelo
arbol = DecisionTreeRegressor(random_state =0)

# ajustamos el modelo
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [158]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.1622776601683795

In [159]:
# max depth

print(arbol.tree_.max_depth)

21


In [160]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [161]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [162]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decission Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,509.794521,442702.972603,665.359281,0.794909,test,Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree I


In [246]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [9,10,11,12], 
        "max_features": [1,2,3],
        "min_samples_split": [20,25,45],
        "min_samples_leaf": [5,15,45]} 

In [247]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch

gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [248]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [9, 10, 11, 12],
                         'max_features': [1, 2, 3],
                         'min_samples_leaf': [5, 15, 45],
                         'min_samples_split': [20, 25, 45]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [249]:
# este método nos esta diciendo que el mejor modelo es aquel que tiene una profundidad de 10, que usa 3 variables predictoras para construir el modelo y que tiene  un min_samples_leaf y un min_samples_split de 10. 
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=9, max_features=3, min_samples_leaf=5,
                      min_samples_split=25)

In [250]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [251]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2


,MAE,MSE,RMSE,R2,set,modelo
0,523.435540,585420.139993,765.127532,0.728793,test,Decision tree II
1,503.321436,459812.195431,678.094533,0.815565,train,Decision tree II


In [252]:
# vamos  a juntar los dataframes de los resultados de los modelos para poder compararlos mejor

df_decTree_cod_results = pd.concat([dt_results1, dt_results2,], axis = 0)
df_decTree_cod_results

,MAE,MSE,RMSE,R2,set,modelo
0,509.794521,442702.972603,665.359281,0.794909,test,Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree I
0,523.435540,585420.139993,765.127532,0.728793,test,Decision tree II
1,503.321436,459812.195431,678.094533,0.815565,train,Decision tree II


In [253]:
# volvemos adefinir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [9,10,11,12], 
        "max_features": [1,2,3],
        "min_samples_split": [20,30,55],
        "min_samples_leaf": [5,15,45]} 

In [254]:
gs_1 = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param,
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 


In [255]:
gs_1.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [9, 10, 11, 12],
                         'max_features': [1, 2, 3],
                         'min_samples_leaf': [5, 15, 45],
                         'min_samples_split': [20, 30, 55]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [256]:
mejor_modelo_1 = gs_1.best_estimator_
mejor_modelo_1

DecisionTreeRegressor(max_depth=11, max_features=3, min_samples_leaf=5,
                      min_samples_split=20)

In [217]:
y_pred_test_dt3 = mejor_modelo_1.predict(x_test)
y_pred_train_dt3 = mejor_modelo_1.predict(x_train)

In [218]:
dt_results3 = metricas(y_test, y_train, y_pred_test_dt3, y_pred_train_dt3, "Decision tree III")
dt_results3

,MAE,MSE,RMSE,R2,set,modelo
0,573.929654,546012.145282,738.926346,0.747049,test,Decision tree III
1,550.614973,534654.432446,731.200679,0.785545,train,Decision tree III


In [220]:
df_decision_results = pd.concat([dt_results1, dt_results2,dt_results3], axis = 0)
df_decision_results


,MAE,MSE,RMSE,R2,set,modelo
0,509.794521,442702.972603,665.359281,0.794909,test,Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree I
0,621.447436,623736.521799,789.769917,0.711042,test,Decision tree II
1,504.566414,461827.338283,679.578795,0.814756,train,Decision tree II
0,573.929654,546012.145282,738.926346,0.747049,test,Decision tree III
1,550.614973,534654.432446,731.200679,0.785545,train,Decision tree III


In [260]:
df_decision_results.to_csv('../data/05_dectre_registrados.csv')